# TEST SAMPLE FOR FINAL BOT BUILT

In [19]:
import utils
import numpy as np
import io
import string
import argparse
from collections import Counter
import re
import json
import sys

# Import required datasets for testing

In [38]:
tContext, tQuestion, tQuestion_id, tAnswerBegin, tAnswerEnd, tAnswerText, maxLenTContext, maxLenTQuestion = splitDatasets(trainData)

In [39]:
with open('dev-v2.0.json') as json_data:
    d = json.load(json_data)
valData = d['data']

In [42]:
te_Context, te_Question, te_Question_id, te_AnswerBegin, te_AnswerEnd, te_AnswerText, te_maxLenTContext, te_maxLenTQuestion = splitDatasets(valData)

In [45]:
vContext, vToken2CharIdx, vContextOriginal, vQuestion, vQuestion_id, maxLenVContext, maxLenVQuestion = splitValDatasets(valData)

Vectorize train and datasets

In [54]:
vX, vXq, vYBegin, vYEnd = vectorizeData(te_Context, te_Question, te_AnswerBegin, te_AnswerEnd, word_index, context_maxlen, question_maxlen)


# Load saved model

In [ ]:
model.load_weights('model_final.h5')

Predictions with test data

In [67]:
predictions = model.predict([vX, vXq], batch_size=128)
print(predictions[0].shape, predictions[1].shape)

(20302, 766) (20302, 766)


Answer Tokens are extracted with answer begin, end and joint 

In [68]:
ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 
for i in range(predictions[0].shape[0]):
	ansBegin[i] = predictions[0][i, :].argmax()
	ansEnd[i] = predictions[1][i, :].argmax()
print(ansBegin.min(), ansBegin.max(), ansEnd.min(), ansEnd.max())
answers = {}
for i in range(len(vQuestion_id)):
    if ansBegin[i] >= len(vContext[i]):
        answers[vQuestion_id[i]] = ""
    elif ansEnd[i] >= len(vContext[i]):
        answers[vQuestion_id[i]] = vContextOriginal[i][vToken2CharIdx[i][ansBegin[i]]:]
    else:
        answers[vQuestion_id[i]] = vContextOriginal[i][vToken2CharIdx[i][ansBegin[i]]:vToken2CharIdx[i][ansEnd[i]]+len(vContext[i][ansEnd[i]])]

0 282 0 284


In [192]:
with io.open('answers_final.json', 'w', encoding='utf-8') as f:
    f.write((json.dumps(answers, ensure_ascii=False)))

# TESTING MODEL

In [69]:
def chatbot(number):
    Q = te_Question[number]
    C = (list(te_Context[number]))
    qid = te_Question_id[number]
    for id,ans in answers.items():   
        if id == qid:
            A= (ans)
    return Q,C,A
    

In [126]:
def print_ans(Q,C,A):
    print("Question")
    print(' '.join(Q))
    print('')
    print('')
    print("Context")
    print( ' '.join(C))
    print('')
    print('')
    print("Answer")
    print(A)       

Example 1

In [128]:
Q,C,A = chatbot(11)
print_ans(Q,C,A)

Question
from which countries did the norse originate ?


Context
the normans ( norman : nourmands ; french : normands ; latin : normanni ) were the people who in the 10th and 11th centuries gave their name to normandy , a region in france . they were descended from norse ( " norman " comes from " norseman " ) raiders and pirates from denmark , iceland and norway who , under their leader rollo , agreed to swear fealty to king charles iii of west francia . through generations of assimilation and mixing with the native frankish and roman-gaulish populations , their descendants would gradually merge with the carolingian-based cultures of west francia . the distinct cultural and ethnic identity of the normans emerged initially in the first half of the 10th century , and it continued to evolve over the succeeding centuries .


Answer
Normandy, a region in France.


Example 2

In [129]:
Q,C,A = chatbot(321)
print_ans(Q,C,A)

Question
what field of computer science analyzes the resource requirements of a specific algorithm isolated unto itself within a given problem ?


Context
closely related fields in theoretical computer science are analysis of algorithms and computability theory . a key distinction between analysis of algorithms and computational complexity theory is that the former is devoted to analyzing the amount of resources needed by a particular algorithm to solve a problem , whereas the latter asks a more general question about all possible algorithms that could be used to solve the same problem . more precisely , it tries to classify problems that can or can not be solved with appropriately restricted resources . in turn , imposing restrictions on the available resources is what distinguishes computational complexity from computability theory : the latter theory asks what kind of problems can , in principle , be solved algorithmically .


Answer
theory. A key


Example 3

In [130]:
Q,C,A = chatbot(444)
print_ans(Q,C,A)

Question
what is the term for a mathematical model that theoretically represents a general computing machine ?


Context
a turing machine is a mathematical model of a general computing machine . it is a theoretical device that manipulates symbols contained on a strip of tape . turing machines are not intended as a practical computing technology , but rather as a thought experiment representing a computing machine—anything from an advanced supercomputer to a mathematician with a pencil and paper . it is believed that if a problem can be solved by an algorithm , there exists a turing machine that solves the problem . indeed , this is the statement of the church–turing thesis . furthermore , it is known that everything that can be computed on other models of computation known to us today , such as a ram machine , conway 's game of life , cellular automata or any programming language can be computed on a turing machine . since turing machines are easy to analyze mathematically , and are be

Example 4

In [131]:
Q,C,A = chatbot(1)
print_ans(Q,C,A)

Question
in what country is normandy located ?


Context
the normans ( norman : nourmands ; french : normands ; latin : normanni ) were the people who in the 10th and 11th centuries gave their name to normandy , a region in france . they were descended from norse ( " norman " comes from " norseman " ) raiders and pirates from denmark , iceland and norway who , under their leader rollo , agreed to swear fealty to king charles iii of west francia . through generations of assimilation and mixing with the native frankish and roman-gaulish populations , their descendants would gradually merge with the carolingian-based cultures of west francia . the distinct cultural and ethnic identity of the normans emerged initially in the first half of the 10th century , and it continued to evolve over the succeeding centuries .


Answer
Normandy, a region in France.


Example 5

In [132]:
Q,C,A = chatbot(20)
print_ans(Q,C,A)

Question
who was the duke in the battle of hastings ?


Context
the norman dynasty had a major political , cultural and military impact on medieval europe and even the near east . the normans were famed for their martial spirit and eventually for their christian piety , becoming exponents of the catholic orthodoxy into which they assimilated . they adopted the gallo-romance language of the frankish land they settled , their dialect becoming known as norman , normaund or norman french , an important literary language . the duchy of normandy , which they formed by treaty with the french crown , was a great fief of medieval france , and under richard i of normandy was forged into a cohesive and formidable principality in feudal tenure . the normans are noted both for their culture , such as their unique romanesque architecture and musical traditions , and for their significant military accomplishments and innovations . norman adventurers founded the kingdom of sicily under roger ii after 

Example 6

In [133]:
Q,C,A = chatbot(20)
print_ans(Q,C,A)

Question
who was the duke in the battle of hastings ?


Context
the norman dynasty had a major political , cultural and military impact on medieval europe and even the near east . the normans were famed for their martial spirit and eventually for their christian piety , becoming exponents of the catholic orthodoxy into which they assimilated . they adopted the gallo-romance language of the frankish land they settled , their dialect becoming known as norman , normaund or norman french , an important literary language . the duchy of normandy , which they formed by treaty with the french crown , was a great fief of medieval france , and under richard i of normandy was forged into a cohesive and formidable principality in feudal tenure . the normans are noted both for their culture , such as their unique romanesque architecture and musical traditions , and for their significant military accomplishments and innovations . norman adventurers founded the kingdom of sicily under roger ii after 

Example 7

In [147]:
Q,C,A = chatbot(266)
print_ans(Q,C,A)

Question
what is norman art 's most well known piece ?


Context
by far the most famous work of norman art is the bayeux tapestry , which is not a tapestry but a work of embroidery . it was commissioned by odo , the bishop of bayeux and first earl of kent , employing natives from kent who were learned in the nordic traditions imported in the previous half century by the danish vikings .


Answer
of Norman art is the Bayeux Tapestry, which is not a tapestry but a work of embroidery. It was commissioned by Odo, the Bishop of Bayeux and first Earl of Kent, employing natives from Kent who were learned in the Nordic traditions imported in the previous half century by the Danish Vikings.


Example 8

In [151]:
Q,C,A = chatbot(420)
print_ans(Q,C,A)

Question
how is the time needed to obtain the solution to a problem calculated ?


Context
to measure the difficulty of solving a computational problem , one may wish to see how much time the best algorithm requires to solve the problem . however , the running time may , in general , depend on the instance . in particular , larger instances will require more time to solve . thus the time required to solve a problem ( or the space required , or any measure of complexity ) is calculated as a function of the size of the instance . this is usually taken to be the size of the input in bits . complexity theory is interested in how algorithms scale with an increase in the input size . for instance , in the problem of finding whether a graph is connected , how much more time does it take to solve a problem for a graph with 2n vertices compared to the time taken for a graph with n vertices ?


Answer
general, depend on the instance. In particular, larger instances will require more time to solv

Example 9

In [169]:
Q,C,A = chatbot(1997)
print_ans(Q,C,A)

Question
how many students were enrolled in public schools in victoria ?


Context
as of august 2010 , victoria had 1,548 public schools , 489 catholic schools and 214 independent schools . just under 540,800 students were enrolled in public schools , and just over 311,800 in private schools . over 61 per cent of private students attend catholic schools . more than 462,000 students were enrolled in primary schools and more than 390,000 in secondary schools . retention rates for the final two years of secondary school were 77 per cent for public school students and 90 per cent for private school students . victoria has about 63,519 full-time teachers .


Answer
under 540,800


Example 10

In [193]:
Q,C,A = chatbot(6005)
print_ans(Q,C,A)

Question
what extinction event might have created some conditions allowing the expansion of the amazon rainforest ?


Context
following the cretaceous–paleogene extinction event , the extinction of the dinosaurs and the wetter climate may have allowed the tropical rainforest to spread out across the continent . from 66–34 mya , the rainforest extended as far south as 45° . climate fluctuations during the last 34 million years have allowed savanna regions to expand into the tropics . during the oligocene , for example , the rainforest spanned a relatively narrow band . it expanded again during the middle miocene , then retracted to a mostly inland formation at the last glacial maximum . however , the rainforest still managed to thrive during these glacial periods , allowing for the survival and evolution of a broad diversity of species .


Answer
Climate fluctuations
